In [3]:
# imports
import sys 
import random
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as colors 
import matplotlib.cbook as cbook 
plt.rcParams.update({'font.size': 12}) 
import mpld3 
import ast
#import ROOT
import uproot3 as ur3
import uproot as ur4
import pandas as pd 
from tqdm import tqdm 
from scipy.optimize import curve_fit 
from scipy import stats
from scipy.signal import lfilter, filtfilt
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
import scipy.integrate as integrate
from lmfit import Model
import threading
from pathlib import Path
from matplotlib import gridspec
import datetime
import matplotlib.ticker as mticker
import time
import pathlib
import awkward as ak
import pprint
import pickle
import struct
from os.path import exists

In [4]:
RootDictonary = {}

In [5]:
def GetRunData(run):
    RootDictonary.update({run:{}})
    file_path = "../FilterData/"+str(run)+"/Analysis.root"

    file = ur4.open(file_path)
    keys_file = file.keys()

    MatchLib = {}
    for key_file in keys_file:
        if "GlobalTimeStamp" in key_file:
            _, key_file_index = key_file.split(";")
            tree = file[key_file]
            LAPPD_ID = tree["LAPPDID"]
            MatchLib[key_file_index] = LAPPD_ID.array()[0]

    for key_file in keys_file:
        _, key_file_index = key_file.split(";")
        tree = file[key_file]
        keys_tree = tree.keys()
        for key_tree in keys_tree:
            branch = tree[key_tree]
            if MatchLib[key_file_index] not in RootDictonary[run]:
                RootDictonary[run].update({MatchLib[key_file_index]:{key_tree:np.array(branch.array())}})
            else:
                RootDictonary[run][MatchLib[key_file_index]][key_tree] = np.array(branch.array())


def PlotSingleBeamPlot(run,savelocation,binwidth):
    timestamp_beam_psec=np.array(RootDictonary[run]["DeltaT_TimeStamp"])

    fig = plt.figure(0, figsize=[15,10], facecolor='white') 
    if run==4248 or run==4249 or run==4250:
        labelval = "R-{}, LAPPD 63 ,entries={}".format(run,len(timestamp_beam_psec))
    else:
        labelval = "R-{}, LAPPD 40 ,entries={}".format(run,len(timestamp_beam_psec))
    plt.hist(timestamp_beam_psec,bins=np.arange(min(timestamp_beam_psec), max(timestamp_beam_psec) + binwidth, binwidth),label=labelval,histtype='stepfilled',alpha=0.8,log=False,color='blue')
    plt.xlabel("time difference between event timestamp and beamgate timestamp in ns")
    plt.ylabel("event/{}ns".format(binwidth))
    plt.legend()
    plt.savefig(savelocation+"SingleBeamPlot_R{}.png".format(run), bbox_inches='tight', pad_inches = 0.5)
    plt.close()

def PlotSinglePPSTimestamp(run,savelocation,binwidth,cutoff):
    timestamp_pps = np.array(RootDictonary[run]["PPSTimeEvolution"])
    timestamp_pps = timestamp_pps[timestamp_pps<cutoff] 

    fig = plt.figure(0, figsize=[15,10], facecolor='white') 
    if run==4248 or run==4249 or run==4250:
        labelval = "R-{}, LAPPD 63, entries={}".format(run,len(timestamp_pps))
    else:
        labelval = "R-{}, LAPPD 40, entries={}".format(run,len(timestamp_pps))
    plt.hist(timestamp_pps,bins=np.arange(min(timestamp_pps), max(timestamp_pps) + binwidth, binwidth),label=labelval,histtype='stepfilled',alpha=0.8,log=True,color='blue')
    plt.xlabel("time difference between pps timestamps in s")
    plt.ylabel("event/{}s".format(binwidth))
    plt.xlim([0,cutoff])
    plt.legend()
    plt.savefig(savelocation+"SinglePPSTimestamp_R{}.png".format(run), bbox_inches='tight', pad_inches = 0.5)
    plt.close()

def GetRunTypePlot(run_list):
    RunsToType = [34,34,39,34,34,34,34]

    EvalRuns = []

    for run in run_list:
        timestamp_pps = np.array(RootDictonary[run]["PPSTimeEvolution"])
        try:
            hist, bin_edges = np.histogram(timestamp_pps,bins=np.arange(min(timestamp_pps), max(timestamp_pps) + 1, 1))
        except:
            print(run)

        try:
            pps_pos = bin_edges[np.where(hist==np.max(hist))]
        except:
            print(run)

        if pps_pos<11 and pps_pos>9:
            EvalRuns.append(34)
        if pps_pos<2 and pps_pos>0:
            EvalRuns.append(39)
        if pps_pos == 20:
            EvalRuns.append(30)

    fig = plt.figure(0, figsize=[15,10], facecolor='white') 
    plt.plot(run_list,RunsToType,'-xb')
    plt.plot(run_list,EvalRuns,'-xr')

def EmergencyBreakInPPS(run,savelocation,binwidth):
    rundata = np.array(RootDictonary[run]["PPS_TimeStamp"])/320e6

    list_of_dataparts = []
    tmp_list = []
    for enum in range(0,len(rundata)-1):
        if rundata[enum+1]<rundata[enum]:
            list_of_dataparts.append(tmp_list)
            tmp_list = []
        else:
            tmp_list.append(rundata[enum])

    fig = plt.figure(0, figsize=[10,10], facecolor='white') 
    if run==4248 or run==4249 or run==4250:
        labelval = "R-{}, LAPPD 63, entries={}".format(run,len(rundata))
    else:
        labelval = "R-{}, LAPPD 40, entries={}".format(run,len(rundata))
    for datapart in list_of_dataparts:
        plt.hist(datapart,bins=np.arange(min(datapart), max(datapart) + binwidth, binwidth),histtype='stepfilled',alpha=0.8,log=True,color='blue')
        plt.xlabel("pps timestamps in s")
        plt.ylabel("event/{}s".format(binwidth))
    #plt.legend(labelval)
    plt.savefig(savelocation+"PPSTimestampHistogram_R{}.png".format(run), bbox_inches='tight', pad_inches = 0.5)
    plt.close()

    
def FindSimilarTimeStamps(run,limit):
    array = RootDictonary[run]["PPS_TimeStamp"]/320e6
    for en1,entry in enumerate(array):
        for en2,loop in enumerate(array):
            if np.fabs(entry-loop)<=limit and en1!=en2:
                print("Entry {} with {} is the same as entry {} with {}".format(en1,entry,en2,loop))


def Plot_PPS_RPi_TimeDifference(run,savelocation,binwidth):
    dt = RootDictonary[run]["PPS_RPi_dt"]
    dt = (dt - dt[0])*1e-9

    fig = plt.figure(0, figsize=[10,10], facecolor='white')
    if run==4248 or run==4249 or run==4250:
        labelval = "R-{}, LAPPD 63, entries={}".format(run,len(dt))
    else:
        labelval = "R-{}, LAPPD 40, entries={}".format(run,len(dt)) 
    plt.hist(dt,bins=np.arange(min(dt), max(dt) + binwidth, binwidth),label=labelval,histtype='stepfilled',alpha=0.8,log=False,color='blue')
    plt.xlabel("Delta t between pps and rpi timestamp in s")
    plt.ylabel("event/{}s".format(binwidth))
    plt.legend()
    plt.savefig(savelocation+"PPS_RPi_DT_R{}.png".format(run), bbox_inches='tight', pad_inches = 0.5)
    plt.close()


def PlotPPSHistogram(run,savelocation,binwidth):
    timestamp_pps = np.array(RootDictonary[run]["PPS_TimeStamp"])/320e6

    for enum in range(0,len(timestamp_pps)-1):
        if(timestamp_pps[enum+1]<timestamp_pps[enum]):
            EmergencyBreakInPPS(run,savelocation,binwidth)
            return

    fig = plt.figure(0, figsize=[10,10], facecolor='white') 
    if run==4248 or run==4249 or run==4250:
        labelval = "R-{}, LAPPD 63, entries={}".format(run,len(timestamp_pps))
    else:
        labelval = "R-{}, LAPPD 40, entries={}".format(run,len(timestamp_pps))
    plt.hist(timestamp_pps,bins=np.arange(min(timestamp_pps), max(timestamp_pps) + binwidth, binwidth),label=labelval,histtype='stepfilled',alpha=0.8,log=True,color='blue')
    plt.xlabel("pps timestamps in s")
    plt.ylabel("event/{}s".format(binwidth))
    plt.legend()
    plt.savefig(savelocation+"PPSTimestampHistogram_R{}.png".format(run), bbox_inches='tight', pad_inches = 0.5)
    plt.close()


In [13]:
run_list = [4344]
for run in run_list:
    try:
        if run not in RootDictonary:
            GetRunData(run)
    except Exception as e:
        print("Failed to load run {} with {}".format(run,e))

    #PlotSingleBeamPlot(run,"../FilterData/{}/".format(run),125)
    PlotSinglePPSTimestamp(run,"../FilterData/{}/".format(run),1,50)
    PlotPPSHistogram(run,"../FilterData/{}/".format(run),10)
    Plot_PPS_RPi_TimeDifference(run,"../FilterData/{}/".format(run),1e-3)

#GetRunTypePlot(run_list)
FindSimilarTimeStamps(run,0)

pprint.pprint(RootDictonary[run_list[0]].keys())

KeyError: 'PPSTimeEvolution'

In [ ]:
#Single run 
run = 4243
binwidth = 125

timestamp_psec=np.array(RootDictonary[run]["Psec_TimeStamp"])*3.125
timestamp_rpi=np.array(RootDictonary[run]["TimeStamp"])/1000
timestamp_beam=np.array(RootDictonary[run]["Beamgate_TimeStamp"])*3.125
timestamp_beam_psec=np.array(RootDictonary[run]["DeltaT_TimeStamp"])

fig = plt.figure(0, figsize=[10,10], facecolor='white') 
if run==4249 or run==4249 or run==4250:
    labelval = "LAPPD 63 ,entries={}".format(len(timestamp_beam_psec))
else:
    labelval = "LAPPD 40 ,entries={}".format(len(timestamp_beam_psec))
plt.hist(timestamp_beam_psec,bins=np.arange(min(timestamp_beam_psec), max(timestamp_beam_psec) + binwidth, binwidth),label=labelval,histtype='stepfilled',alpha=0.8,log=False,color='blue')
plt.xlabel("time difference between events timestamp and beamgate timestamp in ns")
plt.ylabel("event/{}ns".format(binwidth))
plt.legend()

In [ ]:
run = 4356
GetRunData(run)
LAPPD_ID = 0

fig, ax1 = plt.subplots(figsize=[20,10], facecolor='white') 
ax1.plot(RootDictonary[run][LAPPD_ID]["PPSinDelta"])
ax1.set_xlabel("Event #")
ax1.set_ylabel("PPS dt between ACDCs on the same LAPPD in 320MHz clockcycle counts")

ax2 = ax1.twinx()  
ax2.plot(RootDictonary[run][LAPPD_ID]["PPS_TimeStamp"],'xr')
ax2.set_ylabel("PPS timestamp")
plt.title("R"+str(run))
